# pdfplumberの読み取り機能
pdfplumberにおけるPDFの読み取り範囲について知りたい  
Q1. 画像データ内のテキストは読み込めているのか？ ⇒取得できていない  
Q2. テーブル内のテキスト・数字は正しく読めているのか？⇒それなりに取得できている  
Q3. チャンクの大きさは適切か？

In [2]:
import json
import sys
import time
from operator import itemgetter
from pprint import pprint
from typing import Any, Callable, Dict, List

import pdfplumber
import requests
from dotenv import load_dotenv
from langchain import callbacks
from langchain.retrievers import EnsembleRetriever
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter
from pydantic import BaseModel, Field
from sudachipy import dictionary, tokenizer

In [112]:
model = "gpt-4o-mini"
pdf_file_urls = [
    "https://storage.googleapis.com/gg-raggle-public/competitions/617b10e9-a71b-4f2a-a9ee-ffe11d8d64ae/dataset/Financial_Statements_2023.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/617b10e9-a71b-4f2a-a9ee-ffe11d8d64ae/dataset/Hada_Labo_Gokujun_Lotion_Overview.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/617b10e9-a71b-4f2a-a9ee-ffe11d8d64ae/dataset/Shibata_et_al_Research_Article.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/617b10e9-a71b-4f2a-a9ee-ffe11d8d64ae/dataset/V_Rohto_Premium_Product_Information.pdf",
    "https://storage.googleapis.com/gg-raggle-public/competitions/617b10e9-a71b-4f2a-a9ee-ffe11d8d64ae/dataset/Well-Being_Report_2024.pdf",
]

In [95]:
# 戻り値として質問に対する回答を返却してください。
def download_and_load_pdfs(urls: list) -> list:
    """
    PDFファイルをダウンロードして読み込む関数

    Args:
        urls (list): PDFファイルのURLリスト

    Returns:
        documents (list): PDFファイルのテキストデータを含むDocumentオブジェクトのリスト

    Raises:
        Exception: ダウンロードまたは読み込みに失敗した場合に発生する例外

    Examples:
        >>> urls = ["https://example.com/example.pdf"]
        >>> download_and_load_pdfs(urls)
        [Document(page_content="...", metadata={"source": "https://example.com/example.pdf"})]
    """
    try:

        def download_pdf(url, save_path):
            response = requests.get(url)
            if response.status_code == 200:
                with open(save_path, "wb") as f:
                    f.write(response.content)
            else:
                raise Exception(f"Failed to download {url}")

        documents = []

        for i, url in enumerate(urls):
            tmp_path = f"pdf_{i}.pdf"
            download_pdf(url, tmp_path)

            with pdfplumber.open(tmp_path) as pdf:
                full_text = ""
                for page in pdf.pages:
                    text = page.extract_text()
                    if text:
                        full_text += text + "\n"

                documents.append(Document(page_content=full_text, metadata={"source": url}))
        return documents
    except Exception as e:
        raise Exception(f"Error reading {url}: {e}")


def document_transformer(
    docs_list: List[Document], chunk_size: int = 1100, chunk_overlap: int = 100, separator: str = "\n"
) -> List[Document]:
    """ドキュメントリストを LangChain の Document 型に変換し、指定されたサイズで分割する。

    Args:
        docs_list (List[Document]): `content` と `metadata` を持つドキュメントのリスト。
        chunk_size (int): 分割時のチャンクサイズ。
        chunk_overlap (int): 分割時のチャンクのオーバーラップサイズ。
        separator (str): チャンク間の区切り文字。

    Returns:
        List[Document]: 分割後の LangChain の Document 型のリスト。
    """
    # # Step 1: Convert to Document type
    # docs_list_converted = [Document(page_content=doc["content"], metadata=doc["metadata"]) for doc in docs_list]

    # Step 2: Initialize text splitter
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator=separator,
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )

    # Step 3: Split documents
    # doc_splits = text_splitter.split_documents(docs_list_converted)
    doc_splits = text_splitter.split_documents(docs_list)

    return doc_splits


docs = download_and_load_pdfs(pdf_file_urls)
doc_splits = document_transformer(docs)

In [104]:
pprint(docs[0].page_content)

('Resource\n'
 'Selective Laminin-Directed Differentiation of Human\n'
 'Induced Pluripotent Stem Cells into Distinct Ocular\n'
 'Lineages\n'
 'Graphical Abstract Authors\n'
 'ShunShibata,RyuheiHayashi,\n'
 'ToruOkubo,...,AndrewJ.Quantock,\n'
 'KiyotoshiSekiguchi,KohjiNishida\n'
 'Correspondence\n'
 'ryuhei.hayashi@ophthal.med.osaka-u.\n'
 'ac.jp(R.H.),\n'
 'knishida@ophthal.med.osaka-u.ac.jp\n'
 '(K.N.)\n'
 'In Brief\n'
 'Shibataetal.reportthatlamininisoforms\n'
 'differentiallyregulatetheocularcell\n'
 'differentiationfromhiPSCs.Thebinding\n'
 'affinityoflamininandintegrins\n'
 'determinesthenatureofexpandedhiPSC\n'
 'coloniesintermsofcellmotility,cell-cell\n'
 'interactions,andcelldensity,withthe\n'
 'involvementofWntandYAPsignals.\n'
 'Highlights\n'
 'd LN211E8promoteshiPSCdifferentiationintoneuralcrest\n'
 'cellsviaWntactivation\n'
 'd LN332E8promoteshiPSCdifferentiationintocorneal\n'
 'epithelialcells\n'
 'd LN511E8ledtoYAPinactivationandretinaldifferentiationin\n'
 'hiPSCcolonyc

In [97]:
print(doc_splits[0].page_content)

Resource
Selective Laminin-Directed Differentiation of Human
Induced Pluripotent Stem Cells into Distinct Ocular
Lineages
Graphical Abstract Authors
ShunShibata,RyuheiHayashi,
ToruOkubo,...,AndrewJ.Quantock,
KiyotoshiSekiguchi,KohjiNishida
Correspondence
ryuhei.hayashi@ophthal.med.osaka-u.
ac.jp(R.H.),
knishida@ophthal.med.osaka-u.ac.jp
(K.N.)
In Brief
Shibataetal.reportthatlamininisoforms
differentiallyregulatetheocularcell
differentiationfromhiPSCs.Thebinding
affinityoflamininandintegrins
determinesthenatureofexpandedhiPSC
coloniesintermsofcellmotility,cell-cell
interactions,andcelldensity,withthe
involvementofWntandYAPsignals.
Highlights
d LN211E8promoteshiPSCdifferentiationintoneuralcrest
cellsviaWntactivation
d LN332E8promoteshiPSCdifferentiationintocorneal
epithelialcells
d LN511E8ledtoYAPinactivationandretinaldifferentiationin
hiPSCcolonycenters
d Distributionsoflamininsinthedevelopmentaleyemirrored
thehiPSCtypeitinduced
Shibataetal.,2018,CellReports25,1668–1679
November6,2018ª2

In [98]:
print(doc_splits[1].page_content)

is spatiotemporally regulated, especially during development
lead to the selective differentiation of hiPSCs into
(Mineretal.,1997;NguyenandSenior,2006),withseverallam-
different eye-like tissues. Specifically, the 211 iso-
ininisoformsabletosupporthPSCproliferationinundifferenti-
form of the E8 fragment of laminin (LN211E8) pro-
atedstates(Miyazakietal.,2008;Rodinetal.,2010,2014;Xu
motes differentiation into neural crest cells via Wnt
etal.,2001).Truncatedversionsoftheprotein—thelamininE8
activation, whereas LN332E8 promotes differentia- fragments(LNE8)—permitrobustproliferationofhPSCs(Miya-
tion into corneal epithelial cells. The immunohisto- zakietal.,2012),whileotherisoformsareknowntobeeffective
chemical distributions of these laminin isoforms in inthelong-termexpansionorgenerationofspecificsomaticcell
the developing mouse eye mirrors the hiPSC type types from hPSCs (Fusaoka-Nishioka et al., 2011; Kanninen
that was induced in vitro. Moreover, LN511E8 en- etal.,2016;Ohtaetal.,2016;T

In [99]:
print(doc_splits[3].page_content)

sclera.Isoforma2wasexpressedinneuralcrest-derivedtissues
Epithelium + + + (cid:1) +
suchasthecornealstroma,cornealendothelium,choroid,and
(LE) (Epi) (Epi, (Epi,CEpi, sclera, while a3 was expressed in stratified epithelial cells of
CEpi) LE)
the epidermis and corneal epithelium. a4 was expressed in
aAnindicationofzonalformationondifferentlamininisoforms.(++),(+),
cornealendothelium,vitreous,neuroretina,choroid,andsclera.
and((cid:1))indicatelarger,equivalent,orloweramountsofzoneformation
a5wasexpressedinarangeofepithelialtissues,includingthe
thanthatobservedonLN511E8,respectively.
epidermis, corneal epithelium, lens epithelium, retinal pigment
bEpi, epidermis; CEpi, corneal epithelium; CS, corneal stroma; CEnd,
epithelium (RPE), and sclera (Figures 1F and 1G). Collectively,
cornealendothelium;LE,lensepithelium;Chr,Choroid;Scl,Sclera;Vit,
theseresultsindicatedthatdistinctlamininisoformsdifferentially
Vitreous;NR,neuroretina;RPE,retinalpigmentepithelium.
affectautonomousectodermcelldiffer

In [67]:
print("Financial_Statements_2023.pdf")
print(docs[0].page_content)

Financial_Statements_2023.pdf
1/134
EDINET提出書類
ロート製薬株式会社(E00942)
有価証券報告書
【表紙】
【提出書類】 有価証券報告書
【根拠条文】 金融商品取引法第24条第１項
【提出先】 関東財務局長
【提出日】 2024年６月28日
【事業年度】 第88期(自 2023年４月１日 至 2024年３月31日)
【会社名】 ロート製薬株式会社
【英訳名】 ROHTO PHARMACEUTICAL CO.,LTD.
【代表者の役職氏名】 代表取締役社長 チーフオペレーティングオフィサー（COO） 杉 本 雅 史
【本店の所在の場所】 大阪市生野区巽西１丁目８番１号
【電話番号】 大阪 (06)6758−1231(代表)
【事務連絡者氏名】 経理財務部長 中 田 京 子
【最寄りの連絡場所】 ロート製薬株式会社 グランフロント大阪オフィス
(大阪市北区大深町３番１号 グランフロント大阪 タワーＢ29階)
【電話番号】 大阪 (06)6758−1235(代表)
【事務連絡者氏名】 経理財務部長 中 田 京 子
【縦覧に供する場所】 ロート製薬株式会社 東京支社
(東京都港区海岸１丁目２番20号 汐留ビルディング20階)
ロート製薬株式会社 名古屋支店
(名古屋市中村区名駅３丁目19番14号 第２名古屋三交ビル９階)
株式会社東京証券取引所
(東京都中央区日本橋兜町２番１号)
2/134
EDINET提出書類
ロート製薬株式会社(E00942)
有価証券報告書
第一部 【企業情報】
第１ 【企業の概況】
１ 【主要な経営指標等の推移】
(1) 連結経営指標等
回次 第84期 第85期 第86期 第87期 第88期
決算年月 2020年３月 2021年３月 2022年３月 2023年３月 2024年３月
売上高 (百万円) 188,327 181,287 199,646 238,664 270,840
経常利益 (百万円) 22,735 23,910 28,750 35,568 42,434
親会社株主に帰属する
(百万円) 15,410 16,743 21,127 26,377 30,936
当期純利益
包括利益 (百万円) 11,275 19,794 26,810 36,544 43,647
純資産額 (百万円) 140

In [82]:
pprint(doc_splits[0].page_content)

('1/134\n'
 'EDINET提出書類\n'
 'ロート製薬株式会社(E00942)\n'
 '有価証券報告書\n'
 '【表紙】\n'
 '【提出書類】 有価証券報告書\n'
 '【根拠条文】 金融商品取引法第24条第１項\n'
 '【提出先】 関東財務局長\n'
 '【提出日】 2024年６月28日\n'
 '【事業年度】 第88期(自 2023年４月１日 至 2024年３月31日)\n'
 '【会社名】 ロート製薬株式会社\n'
 '【英訳名】 ROHTO PHARMACEUTICAL CO.,LTD.\n'
 '【代表者の役職氏名】 代表取締役社長 チーフオペレーティングオフィサー（COO） 杉 本 雅 史\n'
 '【本店の所在の場所】 大阪市生野区巽西１丁目８番１号\n'
 '【電話番号】 大阪 (06)6758−1231(代表)\n'
 '【事務連絡者氏名】 経理財務部長 中 田 京 子\n'
 '【最寄りの連絡場所】 ロート製薬株式会社 グランフロント大阪オフィス\n'
 '(大阪市北区大深町３番１号 グランフロント大阪 タワーＢ29階)\n'
 '【電話番号】 大阪 (06)6758−1235(代表)\n'
 '【事務連絡者氏名】 経理財務部長 中 田 京 子\n'
 '【縦覧に供する場所】 ロート製薬株式会社 東京支社\n'
 '(東京都港区海岸１丁目２番20号 汐留ビルディング20階)\n'
 'ロート製薬株式会社 名古屋支店\n'
 '(名古屋市中村区名駅３丁目19番14号 第２名古屋三交ビル９階)\n'
 '株式会社東京証券取引所\n'
 '(東京都中央区日本橋兜町２番１号)\n'
 '2/134\n'
 'EDINET提出書類\n'
 'ロート製薬株式会社(E00942)\n'
 '有価証券報告書\n'
 '第一部 【企業情報】\n'
 '第１ 【企業の概況】\n'
 '１ 【主要な経営指標等の推移】\n'
 '(1) 連結経営指標等\n'
 '回次 第84期 第85期 第86期 第87期 第88期')


In [83]:
pprint(doc_splits[1].page_content)

('１ 【主要な経営指標等の推移】\n'
 '(1) 連結経営指標等\n'
 '回次 第84期 第85期 第86期 第87期 第88期\n'
 '決算年月 2020年３月 2021年３月 2022年３月 2023年３月 2024年３月\n'
 '売上高 (百万円) 188,327 181,287 199,646 238,664 270,840\n'
 '経常利益 (百万円) 22,735 23,910 28,750 35,568 42,434\n'
 '親会社株主に帰属する\n'
 '(百万円) 15,410 16,743 21,127 26,377 30,936\n'
 '当期純利益\n'
 '包括利益 (百万円) 11,275 19,794 26,810 36,544 43,647\n'
 '純資産額 (百万円) 140,032 156,612 183,994 215,078 247,036\n'
 '総資産額 (百万円) 215,644 226,149 274,876 309,677 346,175\n'
 '１株当たり純資産額 (円) 608.83 681.71 780.30 916.93 1,078.04\n'
 '１株当たり当期純利益 (円) 67.57 73.39 92.61 115.62 135.61\n'
 '潜在株式調整後\n'
 '(円) 67.36 73.18 92.34 115.29 135.21\n'
 '１株当たり当期純利益\n'
 '自己資本比率 (％) 64.4 68.8 64.8 67.6 71.0\n'
 '自己資本利益率 (％) 11.4 11.7 12.7 13.6 13.6\n'
 '株価収益率 (倍) 21.9 20.1 19.9 24.0 21.9\n'
 '営業活動による\n'
 '(百万円) 19,040 20,008 27,250 30,924 34,245\n'
 'キャッシュ・フロー\n'
 '投資活動による\n'
 '(百万円) △9,405 △10,237 △16,406 △13,176 △16,317\n'
 'キャッシュ・フロー\n'
 '財務活動による\n'
 '(百万円) △1,603 △2,347 3,470 △16,199 △13,784\n'
 'キャッシュ・フロー\n'
 '現金及び現金同

In [5]:
print("Hada_Labo_Gokujun_Lotion_Overview.pdf")
pprint(docs[1].page_content)

Hada_Labo_Gokujun_Lotion_Overview.pdf
('2024/12/06 8:55 肌ラボ 極潤ヒアルロン液 | ロート製薬: 商品情報サイト\n'
 'アイケア スキンケア 外⽪薬 内服薬 検査薬 ⿐・⼝腔ケア ⾷品・サプリメント お役⽴ち情報\n'
 '肌ラボ\n'
 '化粧品\n'
 '肌ラボ 極潤ヒアルロン液\n'
 '発酵技術で⽣まれたうるおい成分 乳酸球菌／ヒア\n'
 'ルロン酸発酵液 配合。やさしいとろみでうるおっ\n'
 'て吸いつくようなもちもち肌に\n'
 '内容量 価格\n'
 'ボトル 170mL オープン価格\n'
 'つめかえ⽤ 170mL オープン価格\n'
 'ポンプ 400mL オープン価格\n'
 '※希望⼩売価格・税込\n'
 '肌ラボ 極潤ヒアルロン液の成分表⽰はこちらをご覧ください。\n'
 '成分表⽰ 販売店検索はこちら\n'
 'JANコード\n'
 '肌ラボ 極潤ヒアルロン液について\n'
 '肌が本当に求めるうるおいを追求した、「極潤」。うるおい成分であるヒアルロン酸にこだわり抜き、必要のない\n'
 'ものはできる限り削ぎ落として、配合成分と容器をシンプルにしました。\n'
 '「極潤ヒアルロン液」は、肌が求めるうるおいを詰めこんだ化粧⽔。4種類のヒアルロン酸（うるおい成分）＊\n'
 'を配合しました。やさしいとろみで、たっぷりうるおい、使うたび、健やかな素肌へ。顔・体・髪の⽑にも。⽼\n'
 '若男⼥、誰でも使えます。\n'
 '弱酸性。無⾹料・無着⾊・オイルフリー・アルコール（エタノール）フリー・パラベンフリー。敏感肌の⽅の協\n'
 '⼒によるパッチテスト済み。（すべての⽅に刺激が起こらないというわけではありません。）\n'
 '本製品の容器には、環境に配慮したバイオマス原料を⼀部使⽤しています。\n'
 '＊：加⽔分解ヒアルロン酸（ナノ化ヒアルロン酸）、アセチルヒアルロン酸Na（スーパーヒアルロン酸）、乳酸球菌／ヒアルロン酸発\n'
 '酵液（乳酸発酵ヒアルロン酸）、ヒアルロン酸Na\n'
 '◆本品は、航空法で定める航空危険物に該当しません。\n'
 '★販売名：ハダラボモイスト化粧⽔d\n'
 '使⽤上の注意\n'
 '＜相談すること＞\n'
 '○肌に異常が⽣じていない

In [6]:
print("Shibata_et_al_Research_Article.pdf")
pprint(docs[2].page_content)

Shibata_et_al_Research_Article.pdf
('Resource\n'
 'Selective Laminin-Directed Differentiation of Human\n'
 'Induced Pluripotent Stem Cells into Distinct Ocular\n'
 'Lineages\n'
 'Graphical Abstract Authors\n'
 'ShunShibata,RyuheiHayashi,\n'
 'ToruOkubo,...,AndrewJ.Quantock,\n'
 'KiyotoshiSekiguchi,KohjiNishida\n'
 'Correspondence\n'
 'ryuhei.hayashi@ophthal.med.osaka-u.\n'
 'ac.jp(R.H.),\n'
 'knishida@ophthal.med.osaka-u.ac.jp\n'
 '(K.N.)\n'
 'In Brief\n'
 'Shibataetal.reportthatlamininisoforms\n'
 'differentiallyregulatetheocularcell\n'
 'differentiationfromhiPSCs.Thebinding\n'
 'affinityoflamininandintegrins\n'
 'determinesthenatureofexpandedhiPSC\n'
 'coloniesintermsofcellmotility,cell-cell\n'
 'interactions,andcelldensity,withthe\n'
 'involvementofWntandYAPsignals.\n'
 'Highlights\n'
 'd LN211E8promoteshiPSCdifferentiationintoneuralcrest\n'
 'cellsviaWntactivation\n'
 'd LN332E8promoteshiPSCdifferentiationintocorneal\n'
 'epithelialcells\n'
 'd LN511E8ledtoYAPinactivationandretinal

In [7]:
print("V_Rohto_Premium_Product_Information.pdf")
pprint(docs[3].page_content)

V_Rohto_Premium_Product_Information.pdf
''


In [68]:
print("Well-Being_Report_2024.pdf")
print(docs[4].page_content)

Well-Being_Report_2024.pdf
ROHTO Well-being Report
2024
ロート製薬 統合レポート
経営理念
「健康」という価値提供を通じて
豊かで幸せな生活を送るための心身の健康に貢献し続けることが
ロートの最大の責務と捉え、その実現のために
長期視点での経営と価値創出に努める
世界の人々を
ロートは、社会の公器としての使命を自覚し、
ロートを取りまくすべての人たちと協働して社会課題を解決し、
これにより得られた便益を共有する
Well-being
「
」
定款 第一章 第二条
に
ロートグループ総合経営ビジョン2030
-Connect for Well-being-
ロート製薬は、創業以来「健康」をコアバリューに、
Well-beingとは身体も心もイキイキとし、 一般用医薬品やスキンケア商品の提供を通じて、
さまざまなライフステージにおいて笑顔あふれる幸せな毎日を過ごすこと 多くの方に身近な「健康」をお届けしてまいりました。
生活者の皆さま一人ひとりの健康寿命が延伸し、
ロート製薬は、世界の人々がWell-beingを実感できる時間が少しでも長くなるように、
生活の質（Quality of Life）が向上することによって、
医薬品、スキンケアに加え、さまざまな事業で
社会全体の経済活動は活性化し、増加する社会保障費も抑制され、
イノベーションを起こし幅広く「つなげていく」
持続的な健康長寿社会の実現につながると考えています。
それを実現するために、社内外の仲間同士を、組織と組織を
当社の存在意義（パーパス）は、
しっかり「つなげていく」
そして、信頼の連鎖の上に人材を育成し、一体感のある組織を作り上げ 世界の人々に商品やサービスを通じて「健康」をお届けすることによって、
人々の更なるWell-beingに「つなげていく」 当社を取り巻くすべての人や社会を「Well-being」へと導き、
明日の世界を元気にする
こと。
これからも、事業活動を通じて世界の人々のWell-beingに貢献するとともに、
健康で幸せに過ごすことができる持続可能な社会の実現を目指してまいります。
1 ROHTO Well-being Report 2024 ROHTO Well-being Report 2024 2
ロートの 価値創造 事業

In [84]:
pprint(doc_splits[-1].page_content)

('10 メンソレータム社 米国 ニューヨーク州 28 ロート・メンソレータム・カンボジア社 カンボジア プノンペン\n'
 '11 ハイドロックス・ラボラトリーズ社 米国 イリノイ州 29 ロート・メンソレータム・ミャンマー社 ミャンマー ヤンゴン\n'
 'アメリカ 連結子会社 9社 12 メンソレータム社・カナダ カナダ オンタリオ州 30 ロート・メンソレータム・ネパール社 ネパール カトマンズ\n'
 'その他 6社 13 メンソレータム社・メキシコ メキシコ メキシコシティ 31 メンソレータム社・イギリス 英国 スコットランド\n'
 '14 ロート・メンソレータム社・ブラジル ブラジル サンパウロ州 ヨーロッパ・ 連結子会社 8社 32 ダクス・コスメティクス社 ポーランド '
 'マゾフシェ県\n'
 '15 オフサルモス社 ブラジル サンパウロ州 アフリカ その他 2社 33 ロート・メンソレータム社・ロシア ロシア モスクワ\n'
 '16 メンソレータム社・アジアパシフィック 中国 香港 34 メンソレータム社・南アフリカ 南アフリカ ケープタウン\n'
 'アジア 17 ロートアドバンスドリサーチ 中国 香港 35 ロート・メンソレータム・ケニア社 ケニア ナイロビ\n'
 '18 メンソレータム社・中国 中国 広東省 オセアニア 連結子会社 1社\n'
 '36 メンソレータム社・オーストラレーシア 豪州 ビクトリア州\n'
 'その他 1社\n'
 '79 ROHTO Well-being Report 2024 ROHTO Well-being Report 2024 80\n'
 'ロート製薬株式会社\n'
 'https://www.rohto.co.jp\n'
 '発行月 2024年8月\n'
 '〒544-8666 大阪市生野区 西1丁目8番1号 Printed in Japan')


# PyMuPDFの読み取りテスト

In [105]:
import fitz
import re


# 戻り値として質問に対する回答を返却してください。
def download_and_load_pdfs_refix(urls: list) -> list:
    """
    PDFファイルをダウンロードして読み込む関数

    Args:
        urls (list): PDFファイルのURLリスト

    Returns:
        documents (list): PDFファイルのテキストデータを含むDocumentオブジェクトのリスト

    Raises:
        Exception: ダウンロードまたは読み込みに失敗した場合に発生する例外

    Examples:
        >>> urls = ["https://example.com/example.pdf"]
        >>> download_and_load_pdfs(urls)
        [Document(page_content="...", metadata={"source": "https://example.com/example.pdf"})]
    """

    try:

        def download_pdf(url, save_path):
            response = requests.get(url)
            if response.status_code == 200:
                with open(save_path, "wb") as f:
                    f.write(response.content)
            else:
                raise Exception(f"Failed to download {url}")

        documents = []

        for i, url in enumerate(urls):
            tmp_path = f"pdf_{i}.pdf"
            download_pdf(url, tmp_path)

            with fitz.open(tmp_path) as pdf:
                full_text = ""
                for page in pdf:
                    text = page.get_text()  # プレーンテキストを取得する
                    if text:
                        full_text += text

                documents.append(Document(page_content=full_text, metadata={"source": url}))

        return documents
    except Exception as e:
        raise Exception(f"Error reading {url}: {e}")


def document_transformer(
    docs_list: List[Document], chunk_size: int = 1100, chunk_overlap: int = 100, separator: str = "\n"
) -> List[Document]:
    """ドキュメントリストを LangChain の Document 型に変換し、指定されたサイズで分割する。

    Args:
        docs_list (List[Document]): `content` と `metadata` を持つドキュメントのリスト。
        chunk_size (int): 分割時のチャンクサイズ。
        chunk_overlap (int): 分割時のチャンクのオーバーラップサイズ。
        separator (str): チャンク間の区切り文字。

    Returns:
        List[Document]: 分割後の LangChain の Document 型のリスト。
    """
    # # Step 1: Convert to Document type
    # docs_list_converted = [Document(page_content=doc["content"], metadata=doc["metadata"]) for doc in docs_list]

    # Step 2: Initialize text splitter
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator=separator,
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )

    # Step 3: Split documents
    # doc_splits = text_splitter.split_documents(docs_list_converted)
    doc_splits = text_splitter.split_documents(docs_list)

    return doc_splits


docs_refix = download_and_load_pdfs_refix(pdf_file_urls)
doc_splits = document_transformer(docs_refix)

In [106]:
pprint(doc_splits[0].page_content)

('Resource\n'
 'Selective Laminin-Directed Differentiation of Human\n'
 'Induced Pluripotent Stem Cells into Distinct Ocular\n'
 'Lineages\n'
 'Graphical Abstract\n'
 'Highlights\n'
 'd LN211E8 promotes hiPSC differentiation into neural crest\n'
 'cells via Wnt activation\n'
 'd LN332E8 promotes hiPSC differentiation into corneal\n'
 'epithelial cells\n'
 'd LN511E8 led to YAP inactivation and retinal differentiation in\n'
 'hiPSC colony centers\n'
 'd Distributions of laminins in the developmental eye mirrored\n'
 'the hiPSC type it induced\n'
 'Authors\n'
 'Shun Shibata, Ryuhei Hayashi,\n'
 'Toru Okubo, ..., Andrew J. Quantock,\n'
 'Kiyotoshi Sekiguchi, Kohji Nishida\n'
 'Correspondence\n'
 'ryuhei.hayashi@ophthal.med.osaka-u.\n'
 'ac.jp (R.H.),\n'
 'knishida@ophthal.med.osaka-u.ac.jp\n'
 '(K.N.)\n'
 'In Brief\n'
 'Shibata et al. report that laminin isoforms\n'
 'differentially regulate the ocular cell\n'
 'differentiation from hiPSCs. The binding\n'
 'afﬁnity of laminin and integrin

In [93]:
print(doc_splits[1].page_content)

世界の人々に商品やサービスを通じて「健康」をお届けすることによって、
当社を取り巻くすべての人や社会を「Well-being」へと導き、
明日の世界を元気にする
　　　　　　　　　　　　　　  　             こと。
これからも、事業活動を通じて世界の人々のWell-beingに貢献するとともに、 
健康で幸せに過ごすことができる持続可能な社会の実現を目指してまいります。
1
2
ROHTO Well-being Report 2024
ROHTO Well-being Report 2024
山田 邦雄
代表取締役会長
会長メッセージ
創業125周年にあたって
本レポートが発行される2024年は当社創業125周年を迎
える節目の年です。100周年となった1999年以降の25年の
間に事業分野の幅を拡げ、売上規模においては5.2倍、時価総
額では14.4倍になるなど、飛躍的成長を遂げることができま
した。これも商品やサービスをご支持くださった生活者の皆
様、従業員の努力やお取引先のご協力とご支援、株主様をはじ
めとするさまざまなステークホルダーの皆様の当社経営に対
するご理解とご支援の賜物だと思っております。この場を借
りて御礼申し上げます。
節目の年にあたり、次の四半世紀の持続的な成長に向けて、
ここに改めてステークホルダーの皆様へ、当社の経営におい
て大切にしている考え方をお伝えし、引き続き当社経営に対
するご理解、ご支援をお願いさせていただきたく思います。
ロートの成長の基軸となる「チャレンジ」
当社の存在意義（パーパス）は、企業理念として定款にも 
記載している通り、世界の人々に商品やサービスを通じて
「健康」をお届けすることによって、当社を取り巻くすべての
個人や社会を「Well-being」に導き、世の中を元気にするこ


In [73]:
print("Financial_Statements_2023.pdf")
print(docs_refix[0].page_content)

Financial_Statements_2023.pdf
 
【表紙】
 
【提出書類】
有価証券報告書
【根拠条文】
金融商品取引法第24条第１項
【提出先】
関東財務局長
【提出日】
2024年６月28日
【事業年度】
第88期(自　2023年４月１日　至　2024年３月31日)
【会社名】
ロート製薬株式会社
【英訳名】
ROHTO　PHARMACEUTICAL　CO.,LTD.
【代表者の役職氏名】
代表取締役社長　チーフオペレーティングオフィサー（COO）　杉　本　雅　史
【本店の所在の場所】
大阪市生野区巽西１丁目８番１号
【電話番号】
大阪　(06)6758－1231(代表)
【事務連絡者氏名】
経理財務部長　中　田　京　子
【最寄りの連絡場所】
ロート製薬株式会社　グランフロント大阪オフィス
　(大阪市北区大深町３番１号　グランフロント大阪　タワーＢ29階)
【電話番号】
大阪　(06)6758－1235(代表)
【事務連絡者氏名】
経理財務部長　中　田　京　子
【縦覧に供する場所】
ロート製薬株式会社　東京支社
　(東京都港区海岸１丁目２番20号 汐留ビルディング20階)
 
ロート製薬株式会社　名古屋支店
　(名古屋市中村区名駅３丁目19番14号第２名古屋三交ビル９階)
 
株式会社東京証券取引所
　(東京都中央区日本橋兜町２番１号)
 
 
EDINET提出書類
ロート製薬株式会社(E00942)
有価証券報告書
  1/134
第一部 【企業情報】
 
第１【企業の概況】
 
１ 【主要な経営指標等の推移】
(1) 連結経営指標等
 
回次
第84期
第85期
第86期
第87期
第88期
決算年月
2020年３月
2021年３月
2022年３月
2023年３月
2024年３月
売上高
(百万円)
188,327
181,287
199,646
238,664
270,840
経常利益
(百万円)
22,735
23,910
28,750
35,568
42,434
親会社株主に帰属する
当期純利益
(百万円)
15,410
16,743
21,127
26,377
30,936
包括利益
(百万円)
11,275
19,794
26,810
36,544
43,647
純資産額
(百万円)
140,032
156,612
183,994


In [71]:
print("Hada_Labo_Gokujun_Lotion_Overview.pdf")
print(docs_refix[1].page_content)

Hada_Labo_Gokujun_Lotion_Overview.pdf
発酵技術で生まれたうるおい成分 乳酸球菌／ヒア ルロン酸発酵液 配合。
やさしいとろみでうるおっ て吸いつくようなもちもち肌に 内容量 価格 ボトル 170mL オープン価格 つめかえ用 170mL オープン価格 ポンプ 400mL オープン価格 ※希望小売価格・税込 肌ラボ 極潤ヒアルロン液の成分表示はこちらをご覧ください。
成分表示 販売店検索はこちら 肌ラボ 極潤ヒアルロン液について 肌が本当に求めるうるおいを追求した、「極潤」。
うるおい成分であるヒアルロン酸にこだわり抜き、必要のない ものはできる限り削ぎ落として、配合成分と容器をシンプルにしました。
「極潤ヒアルロン液」は、肌が求めるうるおいを詰めこんだ化粧水。
4種類のヒアルロン酸（うるおい成分）＊ を配合しました。
やさしいとろみで、たっぷりうるおい、使うたび、健やかな素肌へ。
顔・体・髪の毛にも。
老 若男女、誰でも使えます。
弱酸性。
無香料・無着色・オイルフリー・アルコール（エタノール）フリー・パラベンフリー。
敏感肌の方の協 力によるパッチテスト済み。
（すべての方に刺激が起こらないというわけではありません。
） 本製品の容器には、環境に配慮したバイオマス原料を一部使用しています。
＊：加水分解ヒアルロン酸（ナノ化ヒアルロン酸）、アセチルヒアルロン酸Na（スーパーヒアルロン酸）、乳酸球菌／ヒアルロン酸発 酵液（乳酸発酵ヒアルロン酸）、ヒアルロン酸Na ◆本品は、航空法で定める航空危険物に該当しません。
★販売名：ハダラボモイスト化粧水d 使用上の注意 ＜相談すること＞ ○肌に異常が生じていないかよく注意して使用すること。
使用中、又は使用後日光にあたって、赤み、はれ、か ゆみ、刺激、色抜け（白斑等）や黒ずみ等の異常が現れた時は、使用を中止し、皮フ科専門医等へ相談するこ と。
そのまま使用を続けると症状が悪化することがある。
＜その他使用上の注意＞ ○傷、はれもの、湿疹等、異常のある部位には使用しないこと。
○目に入らないように注意し、入った時はすぐに水又はぬるま湯で洗い流すこと。
なお、異常が残る場合は、眼 科医に相談すること。
肌ラボ 極潤ヒアルロン液に関連する製品 化粧品 肌ラボ 極潤ヒアルロン液 JANコード ア

In [60]:
print("Shibata_et_al_Research_Article.pdf")
# pprint(repr(docs_refix[2].page_content))
pprint(docs_refix[2].page_content)

Shibata_et_al_Research_Article.pdf
('Resource Selective Laminin-Directed Differentiation of Human Induced '
 'Pluripotent Stem Cells into Distinct Ocular Lineages Graphical Abstract '
 'Highlights d LN211E8 promotes hiPSC differentiation into neural crest cells '
 'via Wnt activation d LN332E8 promotes hiPSC differentiation into corneal '
 'epithelial cells d LN511E8 led to YAP inactivation and retinal '
 'differentiation in hiPSC colony centers d Distributions of laminins in the '
 'developmental eye mirrored the hiPSC type it induced Authors Shun Shibata, '
 'Ryuhei Hayashi, Toru Okubo, ..., Andrew J. Quantock, Kiyotoshi Sekiguchi, '
 'Kohji Nishida Correspondence ryuhei.hayashi@ophthal.med.osaka-u. ac.jp '
 '(R.H.), knishida@ophthal.med.osaka-u.ac.jp (K.N.) In Brief Shibata et al. '
 'report that laminin isoforms differentially regulate the ocular cell '
 'differentiation from hiPSCs. The binding afﬁnity of laminin and integrins '
 'determines the nature of expanded hiPSC colonies i

In [61]:
print("V_Rohto_Premium_Product_Information.pdf")
pprint(docs_refix[3].page_content)

V_Rohto_Premium_Product_Information.pdf
''


In [65]:
print("Well-Being_Report_2024.pdf")
print(docs_refix[4].page_content)

Well-Being_Report_2024.pdf
ROHTO Well-being Report ロート製薬 統合レポート2024 ロートグループ総合経営ビジョン2030 -Connect for Well-being- Well-beingとは身体も心もイキイキとし、 さまざまなライフステージにおいて笑顔あふれる幸せな毎日を過ごすこと ロート製薬は、世界の人々がWell-beingを実感できる時間が少しでも長くなるように、 医薬品、スキンケアに加え、さまざまな事業で イノベーションを起こし幅広く「つなげていく」 それを実現するために、社内外の仲間同士を、組織と組織を しっかり「つなげていく」 そして、信頼の連鎖の上に人材を育成し、一体感のある組織を作り上げ 人々の更なるWell-beingに「つなげていく」 経営理念 豊かで幸せな生活を送るための心身の健康に貢献し続けることが ロートの最大の責務と捉え、その実現のために 長期視点での経営と価値創出に努める ロートは、社会の公器としての使命を自覚し、 ロートを取りまくすべての人たちと協働して社会課題を解決し、 これにより得られた便益を共有する 定款 第一章 第二条 「健康」という価値提供を通じて 「Well-being」に 世界の人々を ロート製薬は、創業以来「健康」をコアバリューに、 一般用医薬品やスキンケア商品の提供を通じて、 多くの方に身近な「健康」をお届けしてまいりました。 生活者の皆さま一人ひとりの健康寿命が延伸し、 生活の質（Quality of Life）が向上することによって、 社会全体の経済活動は活性化し、増加する社会保障費も抑制され、 持続的な健康長寿社会の実現につながると考えています。 当社の存在意義（パーパス）は、 世界の人々に商品やサービスを通じて「健康」をお届けすることによって、 当社を取り巻くすべての人や社会を「Well-being」へと導き、 明日の世界を元気にする こと。 これからも、事業活動を通じて世界の人々のWell-beingに貢献するとともに、 健康で幸せに過ごすことができる持続可能な社会の実現を目指してまいります。 1 2 ROHTO Well-being Report 2024 ROHTO Well-being Report 2024 山田 邦雄 代表取締役会長 会長

# easyOCRを使って画像からテキストを抽出

In [4]:
import fitz  # PyMuPDF
import easyocr
import os


def download_pdf(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, "wb") as f:
            f.write(response.content)
    else:
        raise Exception(f"Failed to download {url}")


def pdf_to_images(pdf_path, output_folder="pdf_images"):
    """
    PDFの各ページを画像として保存する関数。

    Args:
        pdf_path (str): PDFファイルのパス。
        output_folder (str): 画像を保存するフォルダ名。

    Returns:
        list: 保存された画像ファイルのパスリスト。
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    pdf_document = fitz.open(pdf_path)
    image_paths = []

    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        pix = page.get_pixmap()
        image_path = os.path.join(output_folder, f"page_{page_num + 1}.png")
        pix.save(image_path)
        image_paths.append(image_path)

    pdf_document.close()
    return image_paths


def extract_text_from_images(image_paths):
    """
    画像からテキストを抽出する関数。

    Args:
        image_paths (list): 画像ファイルのパスリスト。

    Returns:
        str: 抽出されたテキストを結合した文字列。
    """
    reader = easyocr.Reader(["ja", "en"])  # 日本語と英語をサポート
    extracted_text = ""

    for image_path in image_paths:
        print(f"Processing: {image_path}")
        results = reader.readtext(image_path)
        for _, text, _ in results:
            extracted_text += text + "\n"

    return extracted_text


def pdf_text_extraction_pipeline(pdf_path):
    """
    PDFファイルからテキストを抽出するパイプライン。

    Args:
        pdf_path (str): PDFファイルのパス。

    Returns:
        str: PDF全体から抽出されたテキスト。
    """
    print("Converting PDF to images...")
    image_paths = pdf_to_images(pdf_path)

    print("Extracting text from images...")
    extracted_text = extract_text_from_images(image_paths)

    return extracted_text


# 実行例
pdf_path = "/root/git_work/90_adhoc/RagRohtoCompetition/notebook/pdf_3.pdf"
extracted_text = pdf_text_extraction_pipeline(pdf_path)

# 抽出されたテキストを表示
print("Extracted Text:")
print(extracted_text)

Converting PDF to images...
Extracting text from images...
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteProcessing: pdf_images/page_1.png
Processing: pdf_images/page_2.png
Extracted Text:
眞用前にこのに甲車をどすおりみくたさい.支た,む‥なときにオめるよう原@してくたさい. 因・亜
浜恋名'vロートブレミアム』
宮2  三名
ゆ-
ロート史上最高峰処方。
PRENlUM 醐使して番積したつらい療れ目を治す!
国内最多12有効成分配合
歌人の硬れ巨と し,その面因に多アカにアブローア 。
角膜保謹威分高分子コンドロイチン配合
ルーフーーーー・イリトフテルー-リウ
ウ郎す|って保}し 良れ*}@します』
一y -川ゥとい*・こセこル ・・ そerozronいヶw-~ッい川 ‥ アルルー』
とりやばりここちこしたりい
3っル
多ぶ三式
ハつ川
十か一
ひかェにカー ニロし・ロにていき
ロッーアーッアて・て こ-ていり川こ】
〈川・ロ
さ用上山
~ら
がゆサかちらたれんセこけワ・爪町
相盛互ること
町やか上石ッこ
口っにに川ケカリし このに
ゆu[ク川-に下
二ルつル
リールー-一三川川
丘 仮えせにこ相 < たさい:
打たさい
【にけひたいだににていュノ
山W
一門川
無城をにこし
兄り 兄加 かゆみ
ルーーーリーケ
アのたがの田なり  けいい日の町田
_.ざや丞にな_し己と‥
【川~町ににけに .  円〉
ぶの川戸は屋川を中止し
この引をけっこ
〈口は ーこ川さり
馬器ポ愛荒,議良ならない場合
屈志 屈立
1回1二-_15
え回点眼してください』
川ラキ
お_に屋いする_
ぶちしほuしこたりじにりざむて元口をゼ: こと
「「り;すいじはき 市た『 し‥ にさい
りに・にやす
うにに …にラにに か町いヶにやきたて
りにさい
こ先を口にきのに
よとにきりさたないで: にこい はちに戸バ向
~一
jntie_『おににもqしにき"ふり しない
つ‥にきい
・のうと
は ‥ リコーコーー[ニ「-)I…「けレり
日コ十り
り!に川てい さい
ルワ・ニヒローいコに
に百
はたらき
コテトラヒトロ」リン
りワの
の口を山ゆさせこに元町を子すつ
ネナス

In [2]:
import fitz  # PyMuPDF
import easyocr
import os


def pdf_to_images(pdf_path, output_folder="pdf_images", dpi=300):
    """
    PDFの各ページを高解像度の画像として保存する関数。

    Args:
        pdf_path (str): PDFファイルのパス。
        output_folder (str): 画像を保存するフォルダ名。
        dpi (int): 画像解像度（ドット・パー・インチ）。

    Returns:
        list: 保存された画像ファイルのパスリスト。
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    pdf_document = fitz.open(pdf_path)
    image_paths = []

    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        # DPIに基づいた解像度で画像を生成
        matrix = fitz.Matrix(dpi / 72, dpi / 72)
        pix = page.get_pixmap(matrix=matrix)
        image_path = os.path.join(output_folder, f"page_{page_num + 1}.png")
        pix.save(image_path)
        image_paths.append(image_path)

    pdf_document.close()
    return image_paths


def extract_text_from_images(image_paths):
    """
    画像からテキストを抽出する関数。

    Args:
        image_paths (list): 画像ファイルのパスリスト。

    Returns:
        str: 抽出されたテキストを結合した文字列。
    """
    reader = easyocr.Reader(["ja", "en"])  # 日本語と英語をサポート
    extracted_text = ""

    for image_path in image_paths:
        print(f"Processing: {image_path}")
        results = reader.readtext(image_path)
        for _, text, _ in results:
            extracted_text += text + "\n"

    return extracted_text


def pdf_text_extraction_pipeline(pdf_path, dpi=300):
    """
    PDFファイルからテキストを抽出するパイプライン。

    Args:
        pdf_path (str): PDFファイルのパス。
        dpi (int): 画像の解像度（ドット・パー・インチ）。

    Returns:
        str: PDF全体から抽出されたテキスト。
    """
    print("Converting PDF to images...")
    image_paths = pdf_to_images(pdf_path, dpi=dpi)

    print("Extracting text from images...")
    extracted_text = extract_text_from_images(image_paths)

    return extracted_text


# 実行例
pdf_path = "/root/git_work/90_adhoc/RagRohtoCompetition/notebook/pdf_3.pdf"
extracted_text = pdf_text_extraction_pipeline(pdf_path, dpi=300)

# 抽出されたテキストを表示
print("Extracted Text:")
print(extracted_text)

Converting PDF to images...
Extracting text from images...
Processing: pdf_images/page_1.png
Processing: pdf_images/page_2.png
Extracted Text:
使用前にこの説明書を必ずお読みください。また 必要なときに読めるよう保管してください。
園n雨繁
販売名:Vロートプレミアム回
第2類医薬品
V
ロート史上最高峰処方。
PREMIUM `使して馨積したつらい渡れ目を治す!
国内最多12有効成分配合
(基準米内)
現代人の渡れ目を科学し その原因に多角的にアプローチ。
角膜保護成分高分子コンドロイチン配合
(コンドロイチン硫酸エステルナトリウム)
角膜を覆って保護し渡れを改善しますc
※ロート製薬の目薬として最多有効成分配合 *基準とは厚生労働省が承認事務の効率化を図るために定めた医薬品の範囲
どの角度からでもさしやすい
あ
フリー
ワンタッチ式
アングル
スクリュー
ノズルG
キヤップ
開ける時は左に1回カチッと回し閉める時も
簡単にアイケアできて とっても便利です
右に1回カチッと回すだけ。簡単便利です_
使用上の注意
包使用後 次の症状があらわれた場合は副作用の
相談すること
可能性があるので 直ちに使用を中止し この説
1.次の人は使用前に医師;
薬剤師又は
明書を持って医師 薬剤師又は登録販売者にご
登録販売者にご相談ください。
相談ください。
(1)医師の治療を受けている人
関係部位
症状
(P)薬などによりアレルギー症状を起こし
皮ふ
発療・発赤 かゆみ
たことがある人
目
充血 かゆみ はれ しみて痛し)
(3)次の症状のある人:はげしい目の痛み
(4)次の診断を受けた人:緑内障
3.次の場合は使用を中止し、 この説明書を持って
医師 薬剤師又は登録販売者にご相談ください。
{影蔵擬慧艶わ撮ならない場合
用法・用量
1回1~2浦
1日5~6回点眼してください。
用法・用量に関連する注意
(1)過度に使用すると 異常なまぶしさを感じたりかえって充血を摺くこと
がありますので用法 用量を厳守してくださし。
(P)小児に使用させる場合には 保護者の指導監督のもとに使用させて
くださし。
(3)容器の先を目やまぶた まつ毛に触れ

In [107]:
import fitz  # PyMuPDF
import easyocr
import os


def pdf_to_images(pdf_path, output_folder="pdf_images", dpi=300):
    """
    PDFの各ページを高解像度の画像として保存する関数。
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    pdf_document = fitz.open(pdf_path)
    image_paths = []

    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        matrix = fitz.Matrix(dpi / 72, dpi / 72)
        pix = page.get_pixmap(matrix=matrix)
        image_path = os.path.join(output_folder, f"page_{page_num + 1}.png")
        pix.save(image_path)
        image_paths.append(image_path)

    pdf_document.close()
    return image_paths


def extract_text_from_images(image_paths):
    """
    画像からテキストを抽出し、段組み順序を正しく整える関数。
    """
    reader = easyocr.Reader(["ja", "en"])  # 日本語と英語をサポート
    extracted_text = ""

    for image_path in image_paths:
        print(f"Processing: {image_path}")
        results = reader.readtext(image_path, detail=1)  # 座標情報を含む結果を取得

        # 座標情報を基にテキストを並び替え
        sorted_results = sorted(results, key=lambda x: (x[0][1], x[0][0]))  # y座標, x座標でソート
        for _, text, _ in sorted_results:
            extracted_text += text + "\n"

    return extracted_text


def pdf_text_extraction_pipeline(pdf_path, dpi=300):
    """
    PDFファイルからテキストを抽出するパイプライン。
    """
    print("Converting PDF to images...")
    image_paths = pdf_to_images(pdf_path, dpi=dpi)

    print("Extracting text from images...")
    extracted_text = extract_text_from_images(image_paths)

    return extracted_text


# 実行例
pdf_path = "/root/git_work/90_adhoc/RagRohtoCompetition/notebook/pdf_3.pdf"
extracted_text = pdf_text_extraction_pipeline(pdf_path, dpi=300)

# 抽出されたテキストを表示
print("Extracted Text:")
print(extracted_text)

Converting PDF to images...
Extracting text from images...
Processing: pdf_images/page_1.png
Processing: pdf_images/page_2.png
Extracted Text:
フリー
ノズルG
アングル
くださし。
1回1~2浦
でくださし。
相談すること
V
たことがある人
(基準米内)
用法・用量に関連する注意
1.次の人は使用前に医師;
(1)医師の治療を受けている人
(4)次の診断を受けた人:緑内障
登録販売者にご相談ください。
(5)点眼用にのみ使用してください_
どの角度からでもさしやすい
簡単にアイケアできて とっても便利です
(P)薬などによりアレルギー症状を起こし
(3)次の症状のある人:はげしい目の痛み
薬剤師又は
がありますので用法 用量を厳守してくださし。
1日5~6回点眼してください。
用法・用量
目
角膜を覆って保護し渡れを改善しますc
使用上の注意
皮ふ
関係部位
(4)ソフトコンタクトレンズを装着したまま使用しないでくださし)。
相談ください。
国内最多12有効成分配合
キヤップ
スクリュー
ロート史上最高峰処方。
ワンタッチ式
(P)小児に使用させる場合には 保護者の指導監督のもとに使用させて
(3)容器の先を目やまぶた まつ毛に触れさせないでくださし。 〔汚染や異物
(1)過度に使用すると 異常なまぶしさを感じたりかえって充血を摺くこと
混入(目やにやホコリ等)の原因となる〕また 混溜したものは使用しなし)
カチッ
現代人の渡れ目を科学し その原因に多角的にアプローチ。
(コンドロイチン硫酸エステルナトリウム)
症状
販売名:Vロートプレミアム回
※ロート製薬の目薬として最多有効成分配合 *基準とは厚生労働省が承認事務の効率化を図るために定めた医薬品の範囲
使用前にこの説明書を必ずお読みください。また 必要なときに読めるよう保管してください。
発療・発赤 かゆみ
あ
容器の先と
角膜保護成分高分子コンドロイチン配合
目やまつ毛ガ
右に1回カチッと回すだけ。簡単便利です_
開ける時は左に1回カチッと回し閉める時も
充血 かゆみ はれ しみて痛し)
ふれることのないように
医師 薬剤師又は登録販売者にご相談ください。
{影蔵擬

# 読み込み部分の実行コードを改善

## 改修前

In [ ]:
# 戻り値として質問に対する回答を返却してください。
def download_and_load_pdfs(urls: list) -> list:
    """
    PDFファイルをダウンロードして読み込む関数

    Args:
        urls (list): PDFファイルのURLリスト

    Returns:
        documents (list): PDFファイルのテキストデータを含むDocumentオブジェクトのリスト

    Raises:
        Exception: ダウンロードまたは読み込みに失敗した場合に発生する例外

    Examples:
        >>> urls = ["https://example.com/example.pdf"]
        >>> download_and_load_pdfs(urls)
        [Document(page_content="...", metadata={"source": "https://example.com/example.pdf"})]
    """
    try:

        def download_pdf(url, save_path):
            response = requests.get(url)
            if response.status_code == 200:
                with open(save_path, "wb") as f:
                    f.write(response.content)
            else:
                raise Exception(f"Failed to download {url}")

        documents = []

        for i, url in enumerate(urls):
            tmp_path = f"pdf_{i}.pdf"
            download_pdf(url, tmp_path)

            with pdfplumber.open(tmp_path) as pdf:
                full_text = ""
                for page in pdf.pages:
                    text = page.extract_text()
                    if text:
                        full_text += text + "\n"

                documents.append(Document(page_content=full_text, metadata={"source": url}))
        return documents
    except Exception as e:
        raise Exception(f"Error reading {url}: {e}")


def document_transformer(
    docs_list: List[Document], chunk_size: int = 1100, chunk_overlap: int = 100, separator: str = "\n"
) -> List[Document]:
    """ドキュメントリストを LangChain の Document 型に変換し、指定されたサイズで分割する。

    Args:
        docs_list (List[Document]): `content` と `metadata` を持つドキュメントのリスト。
        chunk_size (int): 分割時のチャンクサイズ。
        chunk_overlap (int): 分割時のチャンクのオーバーラップサイズ。
        separator (str): チャンク間の区切り文字。

    Returns:
        List[Document]: 分割後の LangChain の Document 型のリスト。
    """

    # Step 1: Initialize text splitter
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator=separator,
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )

    # Step 2: Split documents
    doc_splits = text_splitter.split_documents(docs_list)

    return doc_splits

# 改修後

- PDFファイルをダウンロードして読み込む
    - URLを指定してPDFをダウンロード
    - テキストを抽出
        - V_Rohto_Premium_Product_Information.pdfは画像データになっているのでeasyocrで抽出
        - それ以外はpymupdfで抽出
- ドキュメントリストを LangChain の Document 型に変換し、指定されたサイズで分割する

In [142]:
def download_and_load_pdfs(urls: list) -> list:
    """
    指定されたURLからPDFファイルをダウンロードし、テキストを抽出する関数。

    この関数は、与えられたPDFファイルのURLリストからファイルをダウンロードし、PyMuPDFまたはEasyOCRを
    使用してPDFファイルからテキストを抽出します。

    Args:
        urls (list): ダウンロード対象のPDFファイルのURLリスト。

    Raises:
        Exception: ダウンロードまたはPDFの読み取りに失敗した場合に発生。

    Returns:
        list: Documentオブジェクトのリスト。
            各Documentにはテキスト（`page_content`）とURLソース（`metadata`）が含まれる。

    Examples:
        >>> urls = ["https://example.com/sample1.pdf", "https://example.com/sample2.pdf"]
        >>> documents = download_and_load_pdfs(urls)
        >>> for doc in documents:
        >>>     print(doc.page_content)

    Note:
        'V_Rohto_Premium_Product_Information.pdf'がすべて画像化されているため
        EasyOCRを適応している。それ以外のPDFファイルはPyMuPDFを適応。
    """

    try:

        def download_pdf(url: str, save_path: str) -> None:
            """
            指定されたURLからPDFファイルをダウンロードしてローカルに保存する関数。

            この関数は、指定されたURLからPDFファイルをダウンロードし、
            ローカルストレージに保存します。HTTPステータスコードが200でない場合は
            例外をスローします。

            Args:
                url (str): ダウンロード元のPDFファイルのURL。
                save_path (str): ダウンロードしたPDFファイルを保存するローカルパス。

            Raises:
                Exception: ダウンロードが失敗した場合（HTTPステータスコードが200以外）。

            Examples:
                >>> download_pdf("https://example.com/sample.pdf", "sample.pdf")
                ファイル "sample.pdf" が現在の作業ディレクトリに保存されます。
            """

            response = requests.get(url)
            if response.status_code == 200:
                with open(save_path, "wb") as f:
                    f.write(response.content)
            else:
                raise Exception(f"Failed to download {url}")

        def clean_text(text):
            # 不要な改行と空白の削除
            # cleaned_text = text.replace("\n", " ").strip()
            # 連続するスペースを単一のスペースに置換
            cleaned_text = re.sub(r"\s+", " ", text)
            return cleaned_text

        def read_pdf_by_pymupdf(pdf_path: str) -> list:
            """
            PyMuPDFを使用してPDFファイルのテキストを抽出する関数。

            この関数は、指定されたPDFファイルを開き、すべてのページから
            テキストを抽出して1つの文字列として結合します。

            Args:
                pdf_path (str): 読み取るPDFファイルのパス。

            Returns:
                str: PDF内の全ページから抽出されたテキストを結合した文字列。

            Examples:
                >>> text = read_pdf_by_pymupdf("example.pdf")
                >>> print(text)
                "ページ1のテキスト\nページ2のテキスト\n..."

            Note:
                - ファイルが存在しない場合や無効なPDFの場合は例外が発生します。
                - Unicodeの特殊文字を含む場合、正しく処理されない場合があります。
            """

            with fitz.open(pdf_path) as pdf:
                full_text = ""
                for page in pdf:
                    text = page.get_text()  # プレーンテキストを取得する
                    if text:
                        full_text += text
            return clean_text(full_text)

        def read_pdf_by_easyocr(pdf_path: str) -> list:
            """
            EasyOCRを使用してPDFファイルの画像からテキストを抽出する関数。

            Args:
                pdf_path (str): 読み取るPDFファイルのパス。

            Returns:
                str: PDF内の全ページから抽出されたテキストを結合した文字列。
            """

            def pdf_to_images(pdf_path: str, output_folder: str = "pdf_images", dpi: int = 300) -> list:
                """
                PDFの各ページを画像ファイルとして保存する関数。

                この関数は指定されたPDFファイルを開き、各ページを指定された解像度（dpi）で
                画像化し、指定したフォルダに保存します。

                Args:
                    pdf_path (str): 画像化する対象のPDFファイルのパス。
                    output_folder (str, optional): 画像ファイルを保存するフォルダのパス。デフォルトは "pdf_images"。
                    dpi (int, optional): 出力画像の解像度（ドット・パー・インチ）。デフォルトは300。

                Returns:
                    list: 保存された画像ファイルのパスリスト。

                Raises:
                    Exception: PDFファイルが存在しない場合や、保存時にエラーが発生した場合。

                Examples:
                    >>> images = pdf_to_images("example.pdf", "output_folder", dpi=200)
                    >>> print(images)
                    ['output_folder/page_1.png', 'output_folder/page_2.png']

                Note:
                    - 出力フォルダが存在しない場合は自動的に作成されます。
                    - 画像はPNG形式で保存されます。
                """

                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)

                pdf_document = fitz.open(pdf_path)
                image_paths = []

                for page_num in range(len(pdf_document)):
                    page = pdf_document[page_num]
                    matrix = fitz.Matrix(dpi / 72, dpi / 72)
                    pix = page.get_pixmap(matrix=matrix)
                    image_path = os.path.join(output_folder, f"page_{page_num + 1}.png")
                    pix.save(image_path)
                    image_paths.append(image_path)

                pdf_document.close()
                return image_paths

            def extract_text_from_images(image_paths):
                """
                画像からテキストを抽出する関数。

                Args:
                    image_paths (list): 画像ファイルのパスリスト。

                Returns:
                    str: 抽出されたテキストを結合した文字列。
                """
                reader = easyocr.Reader(["ja", "en"])  # 日本語と英語をサポート
                extracted_text = ""

                for image_path in image_paths:
                    results = reader.readtext(image_path)
                    for _, text, _ in results:
                        extracted_text += text + "\n"

                return extracted_text

            def pdf_text_extraction_pipeline(pdf_path):
                """
                PDFファイルからテキストを抽出するパイプライン。

                Args:
                    pdf_path (str): PDFファイルのパス。

                Returns:
                    str: PDF全体から抽出されたテキスト。
                """

                image_paths = pdf_to_images(pdf_path)
                extracted_text = extract_text_from_images(image_paths)

                return extracted_text

            return pdf_text_extraction_pipeline(pdf_path)

        documents = []

        for i, url in enumerate(urls):
            tmp_path = f"pdf_{i}.pdf"
            download_pdf(url, tmp_path)

            if i != 3:  # V_Rohto_Premium_Product_Information.pdf のみOCRでテキスト抽出
                full_text = read_pdf_by_pymupdf(tmp_path)
            else:
                full_text = read_pdf_by_easyocr(tmp_path)

            documents.append(Document(page_content=full_text, metadata={"source": url}))

        return documents

    except Exception as e:
        raise Exception(f"Error reading {url}: {e}")


def document_transformer(
    docs_list: List[Document], chunk_size: int = 1100, chunk_overlap: int = 100, separator: str = "\n"
) -> List[Document]:
    """ドキュメントリストを LangChain の Document 型に変換し、指定されたサイズで分割する。

    Args:
        docs_list (List[Document]): `content` と `metadata` を持つドキュメントのリスト。
        chunk_size (int): 分割時のチャンクサイズ。
        chunk_overlap (int): 分割時のチャンクのオーバーラップサイズ。
        separator (str): チャンク間の区切り文字。

    Returns:
        List[Document]: 分割後の LangChain の Document 型のリスト。
    """

    # Step 1: Initialize text splitter
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator=separator,
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )

    # Step 2: Split documents
    doc_splits = text_splitter.split_documents(docs_list)

    return doc_splits


docs = download_and_load_pdfs(pdf_file_urls)
doc_splits = document_transformer(docs)

In [144]:
print(docs[0].page_content)

 【表紙】 【提出書類】 有価証券報告書 【根拠条文】 金融商品取引法第24条第１項 【提出先】 関東財務局長 【提出日】 2024年６月28日 【事業年度】 第88期(自 2023年４月１日 至 2024年３月31日) 【会社名】 ロート製薬株式会社 【英訳名】 ROHTO PHARMACEUTICAL CO.,LTD. 【代表者の役職氏名】 代表取締役社長 チーフオペレーティングオフィサー（COO） 杉 本 雅 史 【本店の所在の場所】 大阪市生野区巽西１丁目８番１号 【電話番号】 大阪 (06)6758－1231(代表) 【事務連絡者氏名】 経理財務部長 中 田 京 子 【最寄りの連絡場所】 ロート製薬株式会社 グランフロント大阪オフィス (大阪市北区大深町３番１号 グランフロント大阪 タワーＢ29階) 【電話番号】 大阪 (06)6758－1235(代表) 【事務連絡者氏名】 経理財務部長 中 田 京 子 【縦覧に供する場所】 ロート製薬株式会社 東京支社 (東京都港区海岸１丁目２番20号 汐留ビルディング20階) ロート製薬株式会社 名古屋支店 (名古屋市中村区名駅３丁目19番14号第２名古屋三交ビル９階) 株式会社東京証券取引所 (東京都中央区日本橋兜町２番１号) EDINET提出書類 ロート製薬株式会社(E00942) 有価証券報告書 1/134 第一部 【企業情報】 第１【企業の概況】 １ 【主要な経営指標等の推移】 (1) 連結経営指標等 回次 第84期 第85期 第86期 第87期 第88期 決算年月 2020年３月 2021年３月 2022年３月 2023年３月 2024年３月 売上高 (百万円) 188,327 181,287 199,646 238,664 270,840 経常利益 (百万円) 22,735 23,910 28,750 35,568 42,434 親会社株主に帰属する 当期純利益 (百万円) 15,410 16,743 21,127 26,377 30,936 包括利益 (百万円) 11,275 19,794 26,810 36,544 43,647 純資産額 (百万円) 140,032 156,612 183,994 215,078 247,036 総資産額 (百万円) 215,644 226,149 274,876 30

In [146]:
pprint(doc_splits[0].page_content)

('【表紙】 【提出書類】 有価証券報告書 【根拠条文】 金融商品取引法第24条第１項 【提出先】 関東財務局長 【提出日】 2024年６月28日 '
 '【事業年度】 第88期(自 2023年４月１日 至 2024年３月31日) 【会社名】 ロート製薬株式会社 【英訳名】 ROHTO '
 'PHARMACEUTICAL CO.,LTD. 【代表者の役職氏名】 代表取締役社長 チーフオペレーティングオフィサー（COO） 杉 本 雅 史 '
 '【本店の所在の場所】 大阪市生野区巽西１丁目８番１号 【電話番号】 大阪 (06)6758－1231(代表) 【事務連絡者氏名】 経理財務部長 中 田 '
 '京 子 【最寄りの連絡場所】 ロート製薬株式会社 グランフロント大阪オフィス (大阪市北区大深町３番１号 グランフロント大阪 タワーＢ29階) '
 '【電話番号】 大阪 (06)6758－1235(代表) 【事務連絡者氏名】 経理財務部長 中 田 京 子 【縦覧に供する場所】 ロート製薬株式会社 '
 '東京支社 (東京都港区海岸１丁目２番20号 汐留ビルディング20階) ロート製薬株式会社 名古屋支店 '
 '(名古屋市中村区名駅３丁目19番14号第２名古屋三交ビル９階) 株式会社東京証券取引所 (東京都中央区日本橋兜町２番１号) EDINET提出書類 '
 'ロート製薬株式会社(E00942) 有価証券報告書 1/134 第一部 【企業情報】 第１【企業の概況】 １ 【主要な経営指標等の推移】 (1) '
 '連結経営指標等 回次 第84期 第85期 第86期 第87期 第88期 決算年月 2020年３月 2021年３月 2022年３月 2023年３月 '
 '2024年３月 売上高 (百万円) 188,327 181,287 199,646 238,664 270,840 経常利益 (百万円) 22,735 '
 '23,910 28,750 35,568 42,434 親会社株主に帰属する 当期純利益 (百万円) 15,410 16,743 21,127 '
 '26,377 30,936 包括利益 (百万円) 11,275 19,794 26,810 36,544 43,647 純資産額 (百万円) '
 '140,032 156,612 183,994 

In [128]:
print(doc_splits[2].page_content)

従業員数
 〔ほか、平均臨時
雇用者数〕
(名)
6,619
6,596
6,866
7,176
7,259
[1,591]
[1,477]
[1,591]
[1,571]
[1,633]
 
(注) １　2021年３月期、2022年３月期及び2023年３月期において、企業結合に係る暫定的な会計処理の確定を行って
おり、2020年３月期、2021年３月期及び2022年３月期に係る主要な経営指標等については、暫定的な会計
処理の確定後の指標等となっております。
２　「収益認識に関する会計基準」（企業会計基準第29号2020年３月31日）等を2022年３月期の期首から適用
しており、2022年３月期以降に係る主要な経営指標等については、当該会計基準等を適用した後の指標等と
なっております。
３　2023年１月１日付で普通株式１株につき２株の割合で株式分割を行っております。2020年３月期の期首に当
該株式分割が行われたと仮定し、１株当たり純資産額、１株当たり当期純利益及び潜在株式調整後１株当た
り当期純利益を算定しております。
 
EDINET提出書類
ロート製薬株式会社(E00942)
有価証券報告書
  2/134
 
(2) 提出会社の経営指標等
 
回次
第84期
第85期
第86期
第87期
第88期
決算年月
2020年３月
2021年３月
2022年３月
2023年３月
2024年３月
売上高
(百万円)
106,477
101,009
101,207
115,355
128,937
経常利益
(百万円)
14,374
12,564
17,516
22,462
26,696
当期純利益
(百万円)
6,978
8,920
13,035
16,568
19,970
資本金
(百万円)
6,504
6,504
6,504
6,504
6,504
発行済株式総数
(株)
118,089,155
